In [ ]:
!pip install sycamore-ai[duckdb]
# DocPrep code uses the Sycamore document ETL library: https://github.com/aryn-ai/sycamore 

In [ ]:
!apt-get install poppler-utils

In [ ]:
import pyarrow.fs
import sycamore
import json
import os
import duckdb
from sycamore.functions.tokenizer import OpenAITokenizer
from sycamore.llms import OpenAIModels, OpenAI
from sycamore.transforms import COALESCE_WHITESPACE
from sycamore.transforms.merge_elements import GreedySectionMerger
from sycamore.transforms.partition import ArynPartitioner
from sycamore.transforms.embed import OpenAIEmbedder
from sycamore.materialize_config import MaterializeSourceMode
from sycamore.utils.pdf_utils import show_pages
from sycamore.transforms.summarize_images import SummarizeImages
from sycamore.context import ExecMode

In [ ]:
# It's best to store API keys in a configuration file or set them as environment variables.  
# For quick testing, you can define them here:
#
# os.environ["ARYN_API_KEY"] = "YOUR_ARYN_API_KEY"
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
# Sycamore uses lazy execution for efficiency, so the ETL pipeline will only execute when running cells with specific functions.
  
paths = ["s3://aryn-public/ntsb/59.pdf"]
# Configure your AWS credentials here if the bucket is private
fsys = pyarrow.fs.S3FileSystem(region="us-east-1", anonymous=True)
# Initialize the Sycamore context
ctx = sycamore.init(ExecMode.LOCAL)
# Set the embedding model and its parameters
model_name = "text-embedding-3-small"
max_tokens = 8191
dimensions = 1536
# Initialize the tokenizer
tokenizer = OpenAITokenizer(model_name)

ds = (
    ctx.read.binary(paths, binary_format="pdf", filesystem=fsys)
    # Partition and extract tables and images
    .partition(partitioner=ArynPartitioner(
        threshold="auto",
        use_ocr=True,
        extract_table_structure=True,
        extract_images=True
    ))
    # Use materialize to cache output. If changing upstream code or input files, change setting from USE_STORED to RECOMPUTE to create a new cache.
    .materialize(path="./materialize/partitioned", source_mode=MaterializeSourceMode.USE_STORED)
    # Merge elements into larger chunks
    .merge(merger=GreedySectionMerger(
      tokenizer=tokenizer,  max_tokens=max_tokens, merge_across_pages=False
    ))
    # Split elements that are too big to embed
    .split_elements(tokenizer=tokenizer, max_tokens=max_tokens)
)

ds.execute()

# Display the first 3 pages after chunking
show_pages(ds, limit=3)


In [ ]:
embedded_ds = (
    # Copy document properties to each Document's sub-elements
    ds.spread_properties(["path", "entity"])
    # Convert all Elements to Documents
    .explode() 
    # Embed each Document. You can change the embedding model. Make your target vector index matches this number of dimensions.
    .embed(embedder=OpenAIEmbedder(model_name=model_name))
)
# To know more about docset transforms, please visit https://sycamore.readthedocs.io/en/latest/sycamore/transforms.html


In [ ]:
db_url = "test-1.db"
table_name = "test_table"
# Execute the write operation to DuckDB
embedded_ds.write.duckdb(
    db_url=db_url,
    table_name=table_name,
    dimensions=dimensions
)

In [ ]:
# Verify data has been loaded using DocSet Query to retrieve chunks
# If you previously used a DuckDB in Colab with a different number of vector dimensions, you may need to restart the runtime.
query = f"SELECT * from {table_name}"
query_docs = ctx.read.duckdb(db_url=db_url, table_name=table_name, query=query)
query_docs.show(show_embedding=False)